In [1]:
import pyttsx3
import speech_recognition as sr
import webbrowser
import datetime
import pywhatkit
import os
import yfinance as yf
import pyjokes

ERROR_NO_ENTIENDO = "No te entiendo"
ERROR_NO_RESULTADOS = "No te entendi o no puedo encontrar resultados"
INFO_ESTOY_ESPERANDO = "Estoy esperando tus ordenes"


class ObtencionVoz(object):
	"""Modulo para obtener la información de voz"""
	def __init__(self, languaje_habla="spanish",lenguaje_reconocimiento="es-MX", tasa_muestreo=150,volume=1):
		super(ObtencionVoz, self).__init__()
		self.languaje_habla = languaje_habla
		self.tasa_muestreo = tasa_muestreo
		self.volume = volume
		self.lenguaje_reconocimiento = lenguaje_reconocimiento

	def speaking(self,message):
		engine = pyttsx3.init()
		engine.setProperty('rate', self.tasa_muestreo)
		engine.setProperty('voice', self.languaje_habla)
		engine.setProperty('volume', self.volume)
		engine.say(message)
		engine.runAndWait()

	def transform(self):
		r = sr.Recognizer()
		with sr.Microphone() as source:
			r.adjust_for_ambient_noise(source)
			r.pause_threshold = 0.8
			said = r.listen(source)
			try:
				q = r.recognize_google(said, language=self.lenguaje_reconocimiento)
				return q
			except sr.UnknownValueError:
				return ERROR_NO_ENTIENDO
			except sr.RequestError:
				return ERROR_NO_RESULTADOS
			except e:
				return INFO_ESTOY_ESPERANDO





In [2]:
#imagenes 
import cv2
import os
import imutils
import numpy as np
#utilerias
import threading
import time
from datetime import datetime
import hashlib


MUESTRAS_NUMERO = 800

class ReconocimientoFacial(object):
	"""Modulo para hacer el reconocimiento facial y la autenticación"""
	def __init__(self, numero_muestras=MUESTRAS_NUMERO):
		super(ReconocimientoFacial, self).__init__()
		self.voz = ObtencionVoz()
		self.numero_muestras = numero_muestras

	def getHash(self):
		return hashlib.md5(datetime.now().__str__().encode("utf-8")).hexdigest()

	def prepare_working_directory(self,personName):
		""" Crea las carpetas para almacenar las imagenes del directorio."""
		dataPath = os.path.join(os.getcwd(),"data")
		personPath = os.path.join(dataPath,personName)
		if not os.path.exists(personPath):
			print('Carpeta creada: ',personPath)
			os.makedirs(personPath)
		return dataPath,personPath

	def get_samples_faces_from_cam(self,personPath):
		"""Realiza la toma de las capturas de la camara."""
		labels = ["seri@","sonriente","riendo","enojad@", "Guino un ojo","Guino el otro ojo","Con lentes","Sin Lentes","Hablando"]
		periodo = MUESTRAS_NUMERO//len(labels)
		index=0
		cap = cv2.VideoCapture(-1)
		faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
		count = 0
		conta = 0 
		speak = True
		while True:
			ret, frame = cap.read()
			if ret == False: break
			frame =  imutils.resize(frame, width=640)
			gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
			auxFrame = frame.copy()
			faces = faceClassif.detectMultiScale(gray,1.3,5)
			for (x,y,w,h) in faces:
				cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
				rostro = auxFrame[y:y+h,x:x+w]
				rostro = cv2.resize(rostro,(150,150),interpolation=cv2.INTER_CUBIC)
				cv2.imwrite(personPath + '/rotro_{}_{}.jpg'.format(self.getHash(),count),rostro)
				count = count + 1
				conta += 1
			if conta == periodo:
				index= (index+1)%(len(labels))
				conta = 0
				speak = True
			if speak:
				speak = False
				hilo1 = threading.Thread(target=self.voz.speaking,args=("Podrias poner tu cara "+labels[index],))
				hilo1.start()
				
			cv2.putText(frame,labels[index]+" count: "+str(count)+" index: "+str(index), (114,430), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
			cv2.imshow('frame',frame)
			k = cv2.waitKey(1)
			if k == 27 or count >= MUESTRAS_NUMERO:
				break
		cap.release()
		cv2.destroyAllWindows() 

	def train_model(self,working_directory,face_recognizer = cv2.face.LBPHFaceRecognizer_create(),model_xml="modeloLBPHFace.xml"):
		dataPath = working_directory[0]
		personPath = working_directory[1]
		peopleList = [directorio for directorio in os.listdir(dataPath) if not directorio.startswith(".")]
		print('Lista de personas: ', peopleList)
		labels = []
		facesData = []
		label = 0
		for nameDir in peopleList:
			personPath = dataPath + '/' + nameDir
			print('Leyendo las imágenes')
			for fileName in os.listdir(personPath):
				labels.append(label)
				facesData.append(cv2.imread(personPath+'/'+fileName,0))
				#image = cv2.imread(personPath+'/'+fileName,0)
				#cv2.imshow('image',image)
				#cv2.waitKey(10)
			label = label + 1
			
		#face_recognizer = cv2.face.EigenFaceRecognizer_create()
		#face_recognizer = cv2.face.FisherFaceRecognizer_create()
		#face_recognizer = cv2.face.LBPHFaceRecognizer_create()
		print("Entrenando...")
		face_recognizer.train(facesData, np.array(labels))
		face_recognizer.write(model_xml)
		print("Modelo almacenado...")

	def reconocimiento_facial(self,working_directory):
		dataPath = working_directory[0]
		personPath = working_directory[1]
		imagePaths = os.listdir(dataPath)
		print('imagePaths=',imagePaths)
		#face_recognizer = cv2.face.EigenFaceRecognizer_create()
		#face_recognizer = cv2.face.FisherFaceRecognizer_create()
		#face_recognizer = cv2.face.LBPHFaceRecognizer_create()
		# Leyendo el modelo
		#face_recognizer.read('modeloEigenFace.xml')
		#face_recognizer.read('modeloFisherFace.xml')
		face_recognizer.read('modeloLBPHFace.xml')
		cap = cv2.VideoCapture(0)
		#cap = cv2.VideoCapture('Video.mp4')

		faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
		while True:
			ret,frame = cap.read()
			if ret == False: break
			gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
			auxFrame = gray.copy()
			faces = faceClassif.detectMultiScale(gray,1.3,5)
			for (x,y,w,h) in faces:
				rostro = auxFrame[y:y+h,x:x+w]
				rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
				result = face_recognizer.predict(rostro)
				cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
				'''
				# EigenFaces
				if result[1] < 5700:
					cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
				else:
					cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
				'''
				'''
				# FisherFace
				if result[1] < 500:
					cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
				else:
					cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
				'''
				# LBPHFace
				if result[1] < 70:
					cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
				else:
					cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
				
			cv2.imshow('frame',frame)
			k = cv2.waitKey(1)
			if k == 27:
				break
		cap.release()
		cv2.destroyAllWindows()






In [3]:

face_recognition = ReconocimientoFacial()

working_directory = face_recognition.prepare_working_directory("david")
#face_recognition.get_samples_faces_from_cam(working_directory[1])
print("Entrenando modelo.....")
#face_recognition.train_model(working_directory,face_recognizer = cv2.face.LBPHFaceRecognizer_create(),model_xml="modeloLBPHFace.xml")


Entrenando modelo.....


## Fase de evalucion del modelo 

Evaluando los modelos 

Modelos a realizar la evaluación

In [5]:
modelos = [cv2.face.EigenFaceRecognizer_create(),cv2.face.FisherFaceRecognizer_create(),cv2.face.LBPHFaceRecognizer_create(),]
rutas_modelos = ['modeloEigenFace.xml','modeloFisherFace.xml','modeloLBPHFace.xml',]

Entrenado los modelos

In [6]:
from multiprocessing  import Pool
def f(x):
    modelo = x[0]
    ruta = x[1]
    face_recognition = ReconocimientoFacial()
    face_recognition.train_model(working_directory,
                             face_recognizer = modelo
                             ,model_xml=ruta)
    return ruta

with Pool(32) as p:
    result = p.map_async(f, [(m,r) for m,r in zip(modelos,rutas_modelos)])
    result.wait()

In [7]:
for modelo, rutas_entrenamiento_guardar in zip(modelos,rutas_modelos):
    print(modelo, rutas_entrenamiento_guardar)
    """
    face_recognition.train_model(working_directory,
                             face_recognizer = modelo
                             ,model_xml=rutas_entrenamiento_guardar)
                             """

< cv2.face.EigenFaceRecognizer 0x7f862b749770> modeloEigenFace.xml
< cv2.face.FisherFaceRecognizer 0x7f862b749c10> modeloFisherFace.xml
< cv2.face.LBPHFaceRecognizer 0x7f862b7496b0> modeloLBPHFace.xml


In [8]:

    face_recognition.train_model(working_directory,
                             face_recognizer = modelos[0]
                             ,model_xml=rutas_modelos[0])

Lista de personas:  ['david']
Leyendo las imágenes
Entrenando...
Modelo almacenado...


Carga de data set de entrenamiento

In [116]:
from tqdm import tqdm
def load_data(dir_path, IMG_SIZE):
    faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    X = []
    yo = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    auxFrame = gray.copy()
                    faces = faceClassif.detectMultiScale(gray,1.3,5)
                    for (x,y,w,h) in faces:
                        rostro = auxFrame[y:y+h,x:x+w]
                        rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
                        X.append(rostro)
                        yo.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels

In [121]:
PATH = "/home/alan/Documents/source_code/python/data"
TRAIN_DIR = (f'{PATH}/Training/')
TEST_DIR = (f'{PATH}/Training/Testing/')
#IMG_SIZE= (48, 48)
IMG_SIZE= (150, 150)

In [122]:
x_train, y_train, train_labels = load_data(TRAIN_DIR, IMG_SIZE)

100%|████████████████████████████████████████████| 6/6 [00:11<00:00,  1.87s/it]

603 images loaded from /home/alan/Documents/source_code/python/data/Training/ directory.


In [123]:

#face_recognizer = cv2.face.EigenFaceRecognizer_create()
#face_recognizer = cv2.face.FisherFaceRecognizer_create()
#face_recognizer = cv2.face.LBPHFaceRecognizer_create()
# Leyendo el modelo
#face_recognizer.read('modeloEigenFace.xml')
#face_recognizer.read('modeloFisherFace.xml')
face_recognizer.read('modeloLBPHFace.xml')

In [124]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('modeloLBPHFace.xml')

In [131]:
entrenamiento_evaluado = [ face_recognizer.predict(i)[1] for i in x_train]

In [132]:
len([i for i in entrenamiento_evaluado if i < 70])

0

In [18]:
imagePath = "/home/alan/Documents/source_code/python/data/"
cascPath = "/home/alan/Documents/source_code/python/data/casc"

In [19]:
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [17]:
faceCascade = cv2.CascadeClassifier(cascPath)
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.cv.CV_HAAR_SCALE_IMAGE
)
for (x, y, w, h) in faces:
     cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)


NameError: name 'cascPath' is not defined

In [12]:
model = cv2.face.LBPHFaceRecognizer_create()

In [13]:
model.predict()

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'predict'
> Overload resolution failed:
>  - face_FaceRecognizer.predict() missing required argument 'src' (pos 1)
>  - face_FaceRecognizer.predict() missing required argument 'src' (pos 1)
